LangChain Expression(LCEL) 언어를 이용하여 생성된 체인에 메모리를 추가하는 방법

In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,           # 문자열 기반이 아닌, ChatPromptTemplate 에서 사용할 수 있는 형태로 반환
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 사람과 대화하기 위해 AI의 도움을 받아야 해."),
    # history : memory_key 의 default 값이 history 입니다.
    MessagesPlaceholder(variable_name='history'),
    ("human", "{question}"),
])

# 파라미터를 꼭 하나 넣어줘야 합니다.
def load_memory(_):
    # 메모리 로드
    return memory.load_memory_variables({})['history']

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm
def invoke_chain(question):
    result = chain.invoke({'question': question})
    memory.save_context(
        {'question': question}, 
        {'output': result.content},
    )

    print(result)


In [2]:
invoke_chain("내 이름은 Morty야!")

content='안녕 Morty! 만나서 반가워. 무엇을 도와드릴까요?'


In [3]:
invoke_chain("내 이름이 뭐야?")

content='당신의 이름은 Morty입니다.'
